In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
PROJ_ROOT = os.pardir

# For Scraping
from bs4 import BeautifulSoup
import requests
import time

# For munging
import re
import json

In [3]:
# Tools to strip HTML
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [4]:
filePath = os.path.join(PROJ_ROOT, "data", "raw", "yale_bios.csv")

In [42]:
data = pd.read_csv(filePath)
data.shape

(8618, 2)

In [43]:
data.columns = ["A", "B"]
stream = data["B"].str.contains("synopsis clearfix")

In [71]:
# Preprocessing to remove NAs
new = []
for i in stream:
    if i == True:
        new.append(i)
    else:
        new.append(False)
stream = new
filtered = data[stream]

In [70]:
def formatBio(row):
    bs = BeautifulSoup(row["B"], 'html.parser')
    rawBio = bs.find(class_="synopsis clearfix")
    bioText = str(rawBio)
    processedBio = strip_tags(bioText)
    
    rawName = bs.find(class_="name")
    processedName = strip_tags(str(rawName))
    # Returns a tuple (to be formatted later)
    return (processedBio, processedName)

In [74]:
formattedBios = filtered.apply(formatBio, axis=1)
formattedBios

KeyboardInterrupt: 

In [73]:
filtered.columns

Index([u'A', u'B'], dtype='object')

In [84]:
names = [x[1] for x in formattedBios]
bios = [re.sub("\n", "", x[0]) for x in formattedBios]

In [33]:
# Introduce the JSON file
jsonPath = os.path.join(PROJ_ROOT, "data", "interim", "players.json")
players = pd.read_json(jsonPath)
players.shape

(8702, 17)

In [36]:
players.columns

Index([u'B/T', u'City', u'Cl.', u'Events', u'High School',
       u'Hometown/High School', u'Hometown/Region', u'Ht.', u'Name',
       u'Name_link', u'No.', u'Pos.', u'Region', u'Weapon', u'Wt.', u'season',
       u'sport'],
      dtype='object')

In [41]:
players["Name"][-20:]
players[""]

981           Dylan Gastel
982       Alexander George
983     William Harrington
984         Nathaniel Howe
985            Aaron Jones
986          Matthew Jones
987       Brendan Kelleher
988             Chris King
989            Kevin Koste
99             Justin Ager
990       Pavle Krivokapic
991             Michael Li
992        Joseph Linfield
993       Ihna Mangundayao
994        Matthew Matejka
995       Nicholas Matthes
996       Raymond O'Mahony
997          Nicolai Parks
998         Jaylen Pittman
999    Gabriel Roy Liguori
Name: Name, dtype: object

In [85]:
# Assemble dataframe
namesBios = {
    "Names": names,
    "Bios": bios
    }

nbDataFrame = pd.DataFrame.from_dict(namesBios)

In [86]:
nbDataFrame.shape
# Reorder
nbDataFrame = nbDataFrame[["Names", "Bios"]]
nbDataFrame.head()

,Names,Bios
0,Ryan Brenner,"2009: Made 22 appearances, including 20 starts..."
1,Joe Castaldi,"2010: Team captain.2009: Made 12 appearances, ..."
2,Adam Straus-Goldfarb,2015-16 (Junior): Three seat of the fourth var...
3,Robert Wulbern,2012-13: Rowed in stroke-seat of the 2V at Eas...
4,Matt Coombs,2006 Backup DB who played in all but one game....


In [90]:
namesBiosPath = os.path.join(PROJ_ROOT, "data", "interim", "yale_names_bios.csv.gz")
nbDataFrame.to_csv(namesBiosPath, compression="gzip")

AttributeError: 'DataFrame' object has no attribute 'value_counts'